<a href="https://colab.research.google.com/github/anubisftw/DSNYC103122/blob/master/Melanoma_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install keras-tuner
!pip install opendatasets
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


In [47]:
import opendatasets as od
import pandas
import tensorflow as tf
import numpy as np
from tensorflow.keras import regularizers
import random
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from tensorflow.keras.layers import Dense, Conv2D, Activation, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import RandomFlip, RandomRotation, Resizing
from tensorflow.keras.models import Sequential
import tensorflow.keras.applications.mobilenet
import os

In [7]:
od.download("https://www.kaggle.com/datasets/riyaelizashaju/skin-disease-image-dataset-balanced/download?datasetVersionNumber=1")

Skipping, found downloaded files in "./skin-disease-image-dataset-balanced" (use force=True to force download)


TF Data for train/val/test datasets

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/skin-disease-image-dataset-balanced/Split_smol/train',
    labels='inferred',
    label_mode='categorical',
    image_size=(256,256),
    batch_size=32,
    shuffle=True,
    seed=42
)
# Create the validation dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/skin-disease-image-dataset-balanced/Split_smol/val',
    labels='inferred',
    label_mode='categorical',
    image_size=(256,256),
    batch_size=32,
    shuffle=True,
    seed=42
)

Found 1849 files belonging to 8 classes.
Found 459 files belonging to 8 classes.


Preparing data augmentation and Mobilenetv2 preprocessor (scaling/resizing)

In [49]:
IMG_SIZE = 256

# create data augmentation sequence
data_augmentating = tf.keras.Sequential([
    Resizing(224, 224, interpolation='bilinear', crop_to_aspect_ratio=False),
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.3),
])

# mobilenetv2 preprocessor
mobilnet_preprocessor = tf.keras.applications.mobilenet_v2.preprocess_input

# define base model
base_model = tf.keras.applications.MobileNetV2(input_shape= (224,224,3), include_top=False, weights='imagenet')
prediction_layer = Dense(8, activation = 'softmax')
global_average_layer = GlobalAveragePooling2D()

In [52]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentating(inputs)
x = mobilnet_preprocessor(x)
x = base_model(x, training=False)
x = Conv2D(8, 1, padding = 'same', activation = 'relu')(x)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [53]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 sequential_10 (Sequential)  (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_12 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 tf.math.subtract_12 (TFOpLa  (None, 224, 224, 3)      0         
 mbda)                                                           
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                           

In [54]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),         # lr = learning_rate
    loss='categorical_crossentropy',   # Multi-class classification
    metrics=['accuracy']  
)

In [56]:
history = model.fit(train_ds, epochs=30, validation_data=validation_ds)

Epoch 1/30
58/58 [==============================] - 20s 313ms/step - loss: 1.3643 - accuracy: 0.5387 - val_loss: 1.4355 - val_accuracy: 0.5142
Epoch 2/30
58/58 [==============================] - 20s 329ms/step - loss: 1.2481 - accuracy: 0.5917 - val_loss: 1.2705 - val_accuracy: 0.5948
Epoch 3/30
58/58 [==============================] - 19s 312ms/step - loss: 1.1758 - accuracy: 0.6198 - val_loss: 1.3523 - val_accuracy: 0.5752
Epoch 4/30
58/58 [==============================] - 20s 322ms/step - loss: 1.0945 - accuracy: 0.6517 - val_loss: 1.2538 - val_accuracy: 0.5708
Epoch 5/30
58/58 [==============================] - 19s 311ms/step - loss: 1.0756 - accuracy: 0.6268 - val_loss: 1.2260 - val_accuracy: 0.6166
Epoch 6/30
58/58 [==============================] - 21s 313ms/step - loss: 0.9567 - accuracy: 0.6798 - val_loss: 1.3036 - val_accuracy: 0.5577
Epoch 7/30
58/58 [==============================] - 20s 314ms/step - loss: 0.9164 - accuracy: 0.7090 - val_loss: 1.2027 - val_accuracy: 0.6340

In [19]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense

In [19]:
tf.keras.applications.mobilenet_v2.preprocess_input

<function keras.applications.mobilenet_v2.preprocess_input(x, data_format=None)>

In [15]:
## model2


model2 = Sequential([
        resize_rescale,
        data_augmentating,
        tf.keras.applications.mobilenet.MobileNet(input_shape=(256,256, 3), include_top=False),
        
        Dropout(0.6),
        Convolution2D(8, (1, 1), padding='valid'),
        Activation('relu'),        # Rectified Linear Unit - Activation Function
        GlobalAveragePooling2D(),  # Pooling Layer
        Activation('softmax')      # Softmax - Activation Function - For Multi Class Classification
    ])

17225924/17225924 [==============================] - 2s 0us/step


In [16]:
model2.compile(
    optimizer=Adam(learning_rate=0.0001),         # lr = learning_rate
    loss='categorical_crossentropy',   # Multi-class classification
    metrics=['accuracy']  
)

In [17]:
history = model2.fit(train_ds, epochs=60, validation_data=validation_ds)

Epoch 1/60


58/58 [==============================] - 47s 382ms/step - loss: 1.6216 - accuracy: 0.4213 - val_loss: 2.9125 - val_accuracy: 0.2353
Epoch 2/60
58/58 [==============================] - 23s 355ms/step - loss: 1.0630 - accuracy: 0.6284 - val_loss: 1.5990 - val_accuracy: 0.4444
Epoch 3/60
58/58 [==============================] - 22s 356ms/step - loss: 0.8577 - accuracy: 0.6955 - val_loss: 1.2723 - val_accuracy: 0.5708
Epoch 4/60
58/58 [==============================] - 26s 423ms/step - loss: 0.6908 - accuracy: 0.7566 - val_loss: 1.1801 - val_accuracy: 0.5882
Epoch 5/60
58/58 [==============================] - 24s 390ms/step - loss: 0.6284 - accuracy: 0.7745 - val_loss: 1.0575 - val_accuracy: 0.6275
Epoch 6/60
58/58 [==============================] - 23s 366ms/step - loss: 0.5303 - accuracy: 0.8177 - val_loss: 1.0969 - val_accuracy: 0.6492
Epoch 7/60
58/58 [==============================] - 22s 354ms/step - loss: 0.4701 - accuracy: 0.8394 - val_loss: 1.1355 - val_accuracy: 0.6471
Epoch 8/60

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt



acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.show()